Combine the detection bounding boxes and keypoints predictions amd save them.

In [ ]:
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.detection.model import RetinaNet
from speed_trapv3.detection.config import Config as DetConfig
from speed_trapv3.keypoints.config import Config as KeyConfig
from speed_trapv3.config import Config
from speed_trapv3.utils import slugify, get_prediction_path
from pathlib import Path
import random
from tqdm import tqdm
import imageio
from speed_trapv3.keypoints.dataset import crop_and_resize,process_keypoints, keypoints_post_inference_processing
import numpy as np
import torchvision.transforms as T
import torch
import cv2
import os
import copy
import json

In [ ]:
multi_model_path = "/code/data/datasets/common_hall/detection-keypoint-inference"
existing_list = os.listdir("/root/.darwin/datasets/sparrow-computing/kj_speedtrap/releases/allv1/annotations")
detection_annotation_files = os.listdir(DetConfig.predictions_directory)
for detection_annotation_file in detection_annotation_files:
    if detection_annotation_file not in existing_list:
        detection_file = open(DetConfig.predictions_directory/detection_annotation_file)
        detection_annotation_content = json.load(detection_file)
        if (KeyConfig.predictions_directory/detection_annotation_file).exists():
            keypoint_file = open(KeyConfig.predictions_directory/detection_annotation_file)
            keypoint_annotation_content = json.load(keypoint_file)
            for key_pair in keypoint_annotation_content:
                for idx, tire in enumerate(KeyConfig.keypoint_names):
                    key_dict = {}
                    x, y = key_pair[idx]
                    key_dict["name"] =  tire
                    key_dict["keypoint"] = {
                        "x": x, "y":y
                    }
                    detection_annotation_content['annotations'].append(key_dict)
        if len(detection_annotation_content["annotations"]) > 0:
            with open(Path(multi_model_path)/detection_annotation_file, "w") as f:
                f.write(json.dumps(detection_annotation_content))
        else:
            print(detection_annotation_file, "Did not have any predictions")